In [ ]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import psutil
import plotly.graph_objs as go
import datetime
import pandas as pd

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment compatibility

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Real-Time System Monitoring Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        dcc.Graph(id='cpu-usage-graph'),
        dcc.Graph(id='memory-usage-graph')
    ], style={'display': 'flex', 'justify-content': 'space-around'}),
    
    html.Div([
        dcc.Graph(id='disk-usage-graph'),
        dcc.Graph(id='network-activity-graph')
    ], style={'display': 'flex', 'justify-content': 'space-around'}),
    
    html.H3("Running Processes"),
    dash_table.DataTable(
        id='process-table',
        columns=[
            {'name': 'PID', 'id': 'pid'},
            {'name': 'Name', 'id': 'name'},
            {'name': 'CPU %', 'id': 'cpu_percent'},
            {'name': 'Memory %', 'id': 'memory_percent'}
        ],
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': 'black', 'color': 'white'},
        style_cell={'textAlign': 'left'},
    ),
    
    dcc.Interval(
        id='interval-component',
        interval=2000,  # Update every 2 seconds
        n_intervals=0
    )
])

# Callback for memory usage
@app.callback(Output('memory-usage-graph', 'figure'), [Input('interval-component', 'n_intervals')])
def update_memory_usage(n):
    mem = psutil.virtual_memory()
    figure = go.Figure(data=[go.Pie(labels=['Used', 'Available'], values=[mem.used, mem.available])])
    figure.update_layout(title='Memory Usage')
    return figure

# Store CPU usage history
cpu_history = []

# Callback for CPU usage
@app.callback(Output('cpu-usage-graph', 'figure'), [Input('interval-component', 'n_intervals')])
def update_cpu_usage(n):
    global cpu_history
    timestamp = datetime.datetime.now().strftime('%H:%M:%S')
    cpu_percent = psutil.cpu_percent()
    cpu_history.append((timestamp, cpu_percent))
    if len(cpu_history) > 20:
        cpu_history.pop(0)
    
    x_vals, y_vals = zip(*cpu_history)
    figure = go.Figure(data=[go.Scatter(x=list(x_vals), y=list(y_vals), mode='lines+markers')])
    figure.update_layout(title='CPU Usage (%)', xaxis_title='Time', yaxis_title='Percentage')
    return figure

# Callback for disk usage
@app.callback(Output('disk-usage-graph', 'figure'), [Input('interval-component', 'n_intervals')])
def update_disk_usage(n):
    disk = psutil.disk_usage('/')
    figure = go.Figure(data=[go.Pie(labels=['Used', 'Free'], values=[disk.used, disk.free])])
    figure.update_layout(title='Disk Usage')
    return figure

# Store previous network activity
previous_net_io = psutil.net_io_counters()

# Callback for network activity
@app.callback(Output('network-activity-graph', 'figure'), [Input('interval-component', 'n_intervals')])
def update_network_activity(n):
    global previous_net_io
    current_net_io = psutil.net_io_counters()
    sent_per_sec = current_net_io.bytes_sent - previous_net_io.bytes_sent
    recv_per_sec = current_net_io.bytes_recv - previous_net_io.bytes_recv
    previous_net_io = current_net_io
    
    figure = go.Figure()
    figure.add_trace(go.Bar(name='Bytes Sent', x=['Network'], y=[sent_per_sec]))
    figure.add_trace(go.Bar(name='Bytes Received', x=['Network'], y=[recv_per_sec]))
    figure.update_layout(title='Network Activity (Bytes per second)', barmode='group')
    return figure

# Callback for process table
@app.callback(Output('process-table', 'data'), [Input('interval-component', 'n_intervals')])
def update_process_table(n):
    processes = []
    for proc in psutil.process_iter(attrs=['pid', 'name', 'cpu_percent', 'memory_percent']):
        processes.append(proc.info)
    df = pd.DataFrame(processes)
    if not df.empty:
        df = df.nlargest(10, 'cpu_percent')  # Show top 10 CPU consuming processes
    return df.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True, port=8060)  # Use an alternative port to avoid conflicts
